In [2]:
from dataset import CarsPCL
from model import PointNetCls

from torch.utils.data import DataLoader
from torch.utils.data import random_split
import torch.optim as optim

from tqdm import tqdm

In [3]:
path = r'Datasets'
blue = lambda x: '\033[94m' + x + '\033[0m'

In [4]:
dataset_back = CarsPCL(path, 'back')
dataset_front = CarsPCL(path, 'front')

In [5]:
# We will have 10 samples per batch
batch_size = 10

In [6]:
# Load back dataset
train_dataset_back, test_dataset_back = random_split(dataset_back, [int(len(dataset_back)*0.8), int(len(dataset_back)*0.2)])

train_loader_back = DataLoader(train_dataset_back, batch_size=batch_size, shuffle=True, num_workers=1)  # tmp
test_loader_back = DataLoader(test_dataset_back, batch_size=batch_size, shuffle=True, num_workers=1)

In [7]:
# Load front dataset
train_dataset_front, test_dataset_front = random_split(dataset_front, [int(len(dataset_front)*0.8), int(len(dataset_front)*0.2)])

train_loader_front = DataLoader(train_dataset_front, batch_size=batch_size, shuffle=True, num_workers=4)  # tmp
test_loader_front = DataLoader(test_dataset_front, batch_size=batch_size, shuffle=True, num_workers=4)

In [8]:
test_loader_mix = DataLoader(test_dataset_front+test_dataset_back, batch_size=batch_size, shuffle=True, num_workers=4)

In [9]:
num_batch = len(train_dataset_back)/batch_size -1

In [10]:
import torch.nn.functional as F

def train (model, loader_train, loader_test, optimizer, scheduler, epochs):
    for epoch in range(epochs):
            scheduler.step()
            for i, data in enumerate(loader_train, 0):  #Train against back
                points, target = data
                target = target[:, 0]
                points = points.transpose(2, 1)
                # points, target = points.cuda(), target.cuda() commented out as I do not have CUDA
                optimizer.zero_grad()
                model = model.train()
                pred, trans, trans_feat = model(points)
                loss = F.nll_loss(pred, target)
                if feature_transform:
                    loss += feature_transform_regularizer(trans_feat) * 0.001
                loss.backward()
                optimizer.step()
                pred_choice = pred.data.max(1)[1]
                correct = pred_choice.eq(target.data).cpu().sum()
                #print('[%d: %d/%d] train loss: %f accuracy: %f' % (epoch, i, num_batch, loss.item(), correct.item() / float(batch_size)))
                if i % 10 == 0:
                    j, data = next(enumerate(loader_test, 0))  # Test against FRONT
                    points, target = data
                    target = target[:, 0]
                    points = points.transpose(2, 1)
                    # points, target = points.cuda(), target.cuda() commented out as I do not have CUDA
                    model = model.eval()
                    pred, _, _ = model(points)
                    loss = F.nll_loss(pred, target)
                    pred_choice = pred.data.max(1)[1]
                    correct = pred_choice.eq(target.data).cpu().sum()
                    #print('[%d: %d/%d] %s loss: %f accuracy: %f' % (epoch, i, num_batch, blue('test'), loss.item(), correct.item()/float(batch_size)))
    return model

In [11]:
def test_only(model, loader_test):
    total_correct = 0
    total_testset = 0
    for i,data in tqdm(enumerate(loader_test, 0)):
        points, target = data
        target = target[:, 0]
        points = points.transpose(2, 1)
        # points, target = points.cuda(), target.cuda() commented out as I do not have CUDA
        model = model.eval()
        pred, _, _ = model(points)
        pred_choice = pred.data.max(1)[1]
        correct = pred_choice.eq(target.data).cpu().sum()
        total_correct += correct.item()
        total_testset += points.size()[0]
    return total_correct/float(total_testset)

### Connect with OMNeT++

In [12]:
import zmq

import numpy as np

In [22]:
# Define back model, its optimizer and scheduler
feature_transform = False
classifier_bk_a = PointNetCls(k = len(dataset_back.classes), feature_transform=feature_transform)
optimizer_bk_a = optim.Adam(classifier_bk_a.parameters(), lr=1e-5, betas=(0.9, 0.999))
scheduler_bk_a = optim.lr_scheduler.StepLR(optimizer_bk_a, step_size=20, gamma=0.5)

classifier_bk_b = PointNetCls(k = len(dataset_back.classes), feature_transform=feature_transform)
optimizer_bk_b = optim.Adam(classifier_bk_b.parameters(), lr=1e-5, betas=(0.9, 0.999))
scheduler_bk_b = optim.lr_scheduler.StepLR(optimizer_bk_b, step_size=20, gamma=0.5)

classifier_bk_c = PointNetCls(k = len(dataset_back.classes), feature_transform=feature_transform)
optimizer_bk_c = optim.Adam(classifier_bk_c.parameters(), lr=1e-5, betas=(0.9, 0.999))
scheduler_bk_c = optim.lr_scheduler.StepLR(optimizer_bk_c, step_size=20, gamma=0.5)

In [23]:
# Define front model, its optimizer and scheduler
feature_transform = False
classifier_ft_a = PointNetCls(k = len(dataset_back.classes), feature_transform=feature_transform)
optimizer_ft_a = optim.Adam(classifier_ft_a.parameters(), lr=1e-5, betas=(0.9, 0.999))
scheduler_ft_a = optim.lr_scheduler.StepLR(optimizer_ft_a, step_size=20, gamma=0.5)

classifier_ft_b = PointNetCls(k = len(dataset_back.classes), feature_transform=feature_transform)
optimizer_ft_b = optim.Adam(classifier_ft_b.parameters(), lr=1e-5, betas=(0.9, 0.999))
scheduler_ft_b = optim.lr_scheduler.StepLR(optimizer_ft_b, step_size=20, gamma=0.5)

classifier_ft_c = PointNetCls(k = len(dataset_back.classes), feature_transform=feature_transform)
optimizer_ft_c = optim.Adam(classifier_ft_c.parameters(), lr=1e-5, betas=(0.9, 0.999))
scheduler_ft_c = optim.lr_scheduler.StepLR(optimizer_ft_c, step_size=20, gamma=0.5)

In [24]:
from collections import OrderedDict
acc_back, acc_front = 0, 0
acc_bk_a_list, acc_bk_b_list, acc_bk_c_list = [], [], []
acc_ft_a_list, acc_ft_b_list, acc_ft_c_list = [], [], []

acc_bk_a_after_list, acc_bk_b_after_list, acc_bk_c_after_list = [], [], []
acc_ft_a_after_list, acc_ft_b_after_list, acc_ft_c_after_list = [], [], []
ii=0
while True:
    # Train back
    print("Train back a")
    classifier_bk_a = train(classifier_bk_a, train_loader_back, test_loader_mix, optimizer_bk_a, scheduler_bk_a, 1)  # bk_a to back
    print("Train back b")
    classifier_bk_b = train(classifier_bk_b, train_loader_back, test_loader_mix, optimizer_bk_b, scheduler_bk_b, 1)
    print("Train back c")
    classifier_bk_c = train(classifier_bk_c, train_loader_back, test_loader_mix, optimizer_bk_c, scheduler_bk_c, 1)
    # Train front
    print("Train front a")
    classifier_ft_a = train(classifier_ft_a, train_loader_front, test_loader_mix, optimizer_ft_a, scheduler_ft_a, 1) # bk_a to front
    print("Train front b")
    classifier_ft_b = train(classifier_ft_b, train_loader_front, test_loader_mix, optimizer_ft_b, scheduler_ft_b, 1)
    print("Train front c")
    classifier_ft_c = train(classifier_ft_c, train_loader_front, test_loader_mix, optimizer_ft_c, scheduler_ft_c, 1)
    
    avgd = [(classifier_bk_a.state_dict()[key]+classifier_ft_a.state_dict()[key] + 
            classifier_bk_b.state_dict()[key]+classifier_ft_b.state_dict()[key] + 
            classifier_bk_c.state_dict()[key]+classifier_ft_c.state_dict()[key])/6 for key in classifier_bk_a.state_dict().keys()]
    
    
    # Nearest neighbor scenario
#     avgd_bk_a_ft_a = [(classifier_bk_a.state_dict()[key]+classifier_ft_a.state_dict()[key])/2 for key in classifier_bk_a.state_dict().keys()]
#     avgd_ft_a_bk_b = [(classifier_ft_a.state_dict()[key]+classifier_bk_b.state_dict()[key])/2 for key in classifier_bk_a.state_dict().keys()]
#     avgd_bk_b_ft_b = [(classifier_bk_b.state_dict()[key]+classifier_ft_b.state_dict()[key])/2 for key in classifier_bk_a.state_dict().keys()]
#     avgd_ft_b_bk_c = [(classifier_ft_b.state_dict()[key]+classifier_bk_c.state_dict()[key])/2 for key in classifier_bk_a.state_dict().keys()]
#     avgd_bk_c_ft_c = [(classifier_bk_c.state_dict()[key]+classifier_ft_c.state_dict()[key])/2 for key in classifier_bk_a.state_dict().keys()]
#     avgd_ft_c_bk_a = [(classifier_ft_c.state_dict()[key]+classifier_bk_a.state_dict()[key])/2 for key in classifier_bk_a.state_dict().keys()]
    
    # Save some percent of parameters, back
    pars_avgd_list_bk_a = []
    for i, key in enumerate(classifier_bk_a.state_dict().keys()):
        pars_avgd_list_bk_a.append((key, avgd[i])) if i>len(classifier_bk_a.state_dict())-np.ceil(len(classifier_bk_a.state_dict())*1) else pars_avgd_list_bk_a.append((key, classifier_bk_a.state_dict()[key]))     
    pars_avgd_list_bk_b = []
    for i, key in enumerate(classifier_bk_b.state_dict().keys()):
        pars_avgd_list_bk_b.append((key, avgd[i])) if i>len(classifier_bk_b.state_dict())-np.ceil(len(classifier_bk_b.state_dict())*1) else pars_avgd_list_bk_b.append((key, classifier_bk_b.state_dict()[key]))   
    pars_avgd_list_bk_c = []
    for i, key in enumerate(classifier_bk_c.state_dict().keys()):
        pars_avgd_list_bk_c.append((key, avgd[i])) if i>len(classifier_bk_c.state_dict())-np.ceil(len(classifier_bk_c.state_dict())*1) else pars_avgd_list_bk_c.append((key, classifier_bk_c.state_dict()[key]))
    # Save some percent of parameters, front
    pars_avgd_list_ft_a = []
    for i, key in enumerate(classifier_ft_a.state_dict().keys()):
        pars_avgd_list_ft_a.append((key, avgd[i])) if i>len(classifier_ft_a.state_dict())-np.ceil(len(classifier_ft_a.state_dict())*1) else pars_avgd_list_ft_a.append((key, classifier_ft_a.state_dict()[key]))      
    pars_avgd_list_ft_b = []
    for i, key in enumerate(classifier_ft_b.state_dict().keys()):
        pars_avgd_list_ft_b.append((key, avgd[i])) if i>len(classifier_ft_b.state_dict())-np.ceil(len(classifier_ft_b.state_dict())*1) else pars_avgd_list_ft_b.append((key, classifier_ft_b.state_dict()[key])) 
    pars_avgd_list_ft_c = []
    for i, key in enumerate(classifier_ft_c.state_dict().keys()):
        pars_avgd_list_ft_c.append((key, avgd[i])) if i>len(classifier_ft_c.state_dict())-np.ceil(len(classifier_ft_c.state_dict())*1) else pars_avgd_list_ft_c.append((key, classifier_ft_c.state_dict()[key]))
    
    # Test back
    acc_bk_a = test_only(classifier_bk_a, test_loader_mix) 
    print("Accuracy before merging back a ", acc_bk_a)
    acc_bk_a_list.append(acc_bk_a)
    acc_bk_b = test_only(classifier_bk_b, test_loader_mix)
    print("Accuracy before merging back b ", acc_bk_b)
    acc_bk_b_list.append(acc_bk_b)
    acc_bk_c = test_only(classifier_bk_c, test_loader_mix)
    print("Accuracy before merging back c ", acc_bk_c)
    acc_bk_c_list.append(acc_bk_c)
    # Test front
    acc_ft_a = test_only(classifier_ft_a, test_loader_mix)
    print("Accuracy before merging front a ", acc_ft_a)
    acc_ft_a_list.append(acc_ft_a)
    acc_ft_b = test_only(classifier_ft_b, test_loader_mix)
    print("Accuracy before merging front b ", acc_ft_b)
    acc_ft_b_list.append(acc_ft_b)
    acc_ft_c = test_only(classifier_ft_c, test_loader_mix)
    print("Accuracy before merging front c ", acc_ft_c)
    acc_ft_c_list.append(acc_ft_c)

    
    ii+=1
    print(f"Epoch {ii}")
    if ii == 13:
        break
    # Connect to OMNeT ++
    print("Sending to OMNeT++")
    context = zmq.Context()
    socket = context.socket(zmq.REQ)
    port = "5555"
    socket.connect("tcp://localhost:%s" % port)

    message_from_python = ['0'.encode(), str(classifier_bk_a.state_dict()).encode(),
                           '1'.encode(), str(classifier_ft_a.state_dict()).encode(), 
                           '2'.encode(), str(classifier_bk_b.state_dict()).encode(),
                           '3'.encode(), str(classifier_ft_b.state_dict()).encode(),
                           '4'.encode(), str(classifier_bk_c.state_dict()).encode(),
                           '5'.encode(), str(classifier_ft_c.state_dict()).encode(), ]

    socket.send_multipart(message_from_python)
    message = socket.recv_multipart()
    
    # Save parameters back
    pars_avgd_bk_a = OrderedDict(pars_avgd_list_bk_a)
    pars_avgd_bk_b = OrderedDict(pars_avgd_list_bk_b)
    pars_avgd_bk_c = OrderedDict(pars_avgd_list_bk_c)
    # Save parameters front
    pars_avgd_ft_a = OrderedDict(pars_avgd_list_ft_a)
    pars_avgd_ft_b = OrderedDict(pars_avgd_list_ft_b)
    pars_avgd_ft_c = OrderedDict(pars_avgd_list_ft_c)
    # Load parameters back
    classifier_bk_a.load_state_dict(pars_avgd_bk_a)
    classifier_bk_b.load_state_dict(pars_avgd_bk_b)
    classifier_bk_c.load_state_dict(pars_avgd_bk_c)
    # Load parameters front
    classifier_ft_a.load_state_dict(pars_avgd_ft_a)
    classifier_ft_b.load_state_dict(pars_avgd_ft_b)
    classifier_ft_c.load_state_dict(pars_avgd_ft_c)
    
    # Test new parameters back
    acc_bk_a_after = test_only(classifier_bk_a, test_loader_mix)
    print("Accuracy after merging back a ", acc_bk_a_after)
    acc_bk_a_after_list.append(acc_bk_a_after)
    acc_bk_b_after = test_only(classifier_bk_b, test_loader_mix)
    print("Accuracy after merging back b ", acc_bk_b_after)
    acc_bk_b_after_list.append(acc_bk_b_after)
    acc_bk_c_after = test_only(classifier_bk_c, test_loader_mix)
    print("Accuracy after merging back c ", acc_bk_c_after)
    acc_bk_c_after_list.append(acc_bk_c_after)
    # Test new parameters front
    acc_ft_a_after = test_only(classifier_ft_a, test_loader_mix)
    print("Accuracy after merging front a ", acc_ft_a_after)
    acc_ft_a_after_list.append(acc_ft_a_after)
    acc_ft_b_after = test_only(classifier_ft_b, test_loader_mix)
    print("Accuracy after merging front b ", acc_ft_b_after)
    acc_ft_b_after_list.append(acc_ft_b_after)
    acc_ft_c_after = test_only(classifier_ft_c, test_loader_mix)
    print("Accuracy after merging front c ", acc_ft_c_after)
    acc_ft_c_after_list.append(acc_ft_c_after)
  

Train back a
Train back b
Train back c
Train front a
Train front b
Train front c


15it [00:01, 14.70it/s]


Accuracy before merging back a  0.0


15it [00:00, 15.90it/s]

Accuracy before merging back b  0.04666666666666667


...


15it [00:00, 15.51it/s]

Accuracy after merging front b  0.0



15it [00:00, 15.31it/s]

Accuracy after merging front c  0.0
Train back a


KeyboardInterrupt: 